# BOKEH Visualization notebook
The notebook contains two visualizations featuring 
1. Slider
2. Hover tool

It takes three datasets for from gapminder.

1. [Total energy generation](http://docs.google.com/spreadsheet/pub?key=pyj6tScZqmEehRG-9mMHYdg&output=xlsx)
2. [Total energy consumption](http://docs.google.com/spreadsheet/pub?key=tEu78F4acf0u6MRyhg5-9qQ&output=xlsx)
3. [GDP per capita](http://docs.google.com/spreadsheet/pub?key=0AkBd6lyS3EmpdHo5S0J6ekhVOF9QaVhod05QSGV4T3c&output=xlsx)

In [55]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
from bokeh.plotting import figure
from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import CustomJS, Slider, HoverTool,ColumnDataSource
from bokeh.layouts import row
from bokeh.charts import Scatter

In [56]:
#using pandas to read all three datasets from excel files and taking an inner join for rows and columns to maintain consistency of data along all three datasets
produceTotal = pd.read_excel("Electricity Generation.xls.xlsx")
produceTotal=produceTotal.set_index('Electricity generation total (kilowatt-hours)')
produceTotal.columns = produceTotal.columns.astype(str)
country_list=produceTotal.index.tolist()
year_list=produceTotal.columns.tolist()
year_list=[str(i) for i in year_list]

consumeTotalRaw = pd.read_excel("Indicator_Electricity consumption total.xlsx")
consumeTotalRaw=consumeTotalRaw.set_index('Electricity consumption, total (kWh)')
consumeTotalRaw=consumeTotalRaw.loc[:,year_list]
consumeTotal=consumeTotalRaw.loc[country_list]

dFrame=produceTotal.join(consumeTotal,how='inner',lsuffix='_produce',rsuffix='_consume')
tempFrame = dFrame[['1990_produce','1990_consume']].copy()
dFrame['current_x']=dFrame['1990_produce']
dFrame['current_y']=dFrame['1990_consume']

gdpRaw = pd.read_excel("gdp_data.xlsx")
gdpRaw=gdpRaw.set_index('GDP (constant 2000 US$)')
gdpRaw.columns = gdpRaw.columns.astype(str)
gdp=gdpRaw.loc[country_list]

dFrame2 = produceTotal.join(gdp,how='inner',lsuffix='_produce',rsuffix='_gdp')
tempFrame = dFrame2[['1990_produce','1990_gdp']].copy()
dFrame2['current_x']=dFrame2['1990_produce']
dFrame2['current_y']=dFrame2['1990_gdp']

In [57]:
#sending the output to the notebook
output_notebook()
source=ColumnDataSource(dFrame)
source.add(dFrame.index,"index")

#Setting up hover tooltips
hover = HoverTool(tooltips=[
    ("Country", "@index"),
    ("(x,y)", "(@current_x, @current_y)")
])

#defining the figure object then adding a circle plot to it
fig = figure(plot_width=500, plot_height=500, tools=[hover,'wheel_zoom','box_zoom','box_select','reset'],title="Energy consumption vs. Production",x_axis_type="log",y_axis_type="log")
plt = fig.circle(x='current_x', y='current_y',size=10,color='blue',alpha=0.3,source=source)
fig.xaxis.axis_label= "Total Energy Production in kWh (Log scale)"
fig.yaxis.axis_label= "Total Energy Consumption in kWh (Log scale)"

#javaScript callback function to invoke the python update function constantly
callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var year = cb_obj.value
    data['current_x']=data[year.toString() + "_produce"]
    data['current_y']=data[year.toString() + "_consume"]
    source.trigger("change")
""")

#creating slider object instance and passing the callback function to it
slider = Slider(start=1990, end=2008, value=1990, step=1, title="year")
slider.js_on_change('value', callback)

#Displaying the plot and the slider in a row layout
show(row(fig, slider))

Loading BokehJS ...

In [58]:
#sending the output to the notebook
source2 = ColumnDataSource(dFrame2)
source2.add(dFrame2.index,"index")

#Setting up hover tooltips
hover2 = HoverTool(tooltips=[
    ("Country", "@index"),
    ("(x,y)", "(@current_x, @current_y)")
])

#defining the figure object then adding a circle plot to it
fig2 = figure(plot_width=500, plot_height=500, tools=[hover2,'wheel_zoom','box_zoom','box_select','reset'],title="Energy consumption vs. Production",x_axis_type="log",y_axis_type="log")
plt2 = fig2.circle(x='current_x', y='current_y',size=10,color='firebrick',alpha=0.3,source=source2)
fig2.xaxis.axis_label= "Total Energy Production in kWh (Log scale)"
fig2.yaxis.axis_label= "Total GDP"


#javaScript callback function to invoke the python update function constantly
callback2 = CustomJS(args=dict(source=source2), code = """
    var data = source.data
    var year = cb_obj.value
    data['current_x'] = data[year.toString() + "_produce"]
    data['current_y'] = data[year.toString() + "_gdp"]
    source.trigger("change")
""")


#creating slider object instance and passing the callback function to it
slider2 = Slider(start=1990, end=2008, value=1990, step=1, title="year")
slider2.js_on_change('value', callback2)

#Displaying the plot and the slider in a row layout
show(row(slider2,fig2))